# L3: Retry-based Structured Output

In [2]:
import os
from utils import get_together_api_key

In [3]:
import instructor
from openai import OpenAI

# Instantiate the client
together_client = OpenAI(
    base_url=f"{os.getenv('DLAI_TOGETHER_API_BASE', 'https://api.together.xyz')}/v1",
    api_key=get_together_api_key()
)

In [4]:
response = together_client.chat.completions.create(
    model='meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo',
    # model="gpt-4o-mini",
    messages = [{
        "role": "user", 
        "content": 'sup'
    }]
)

response.choices[0].message.content

"Sup back at you. How's your day going so far?"

## Adding `instructor`

In [5]:
# Wrap together with the instructor client
instructor_client = instructor.from_openai(together_client)

In [6]:
from pydantic import BaseModel

class Greeting(BaseModel):
    hello: str

In [7]:
response = instructor_client.chat.completions.create(
    model='meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo',
    # model="gpt-4o-mini",
    messages = [{
        "role": "user", 
        "content": 'sup'
    }], 
    # Note: OpenAI uses response_format, instructor
    #       uses response_model!
    response_model=Greeting
)

response

Greeting(hello='sup')

## Defining a calendar event

In [8]:
from pydantic import Field
from datetime import date
from typing import List

# Person only has a name, but we can easily extend it with other fields
class Person(BaseModel):
    name: str

class CalendarEvent(BaseModel):
    name: str
    
    # not supported by OpenAI. We want a format like 2025-01-30
    date: date 
    participants: List[Person]

    address_number: str
    street_name: str
    city_name: str

    # Inline regex patterns not supported by OpenAI restrict state code 
    # to be two capital letters (OpenAI does not support pattern fields)
    state_code: str = Field(pattern=r'[A-Z]{2}')

    # Zip code must be five digits
    zip_code: str = Field(pattern=r'\d{5}') 


In [9]:
event_description = """
Alice and Bob are going to a science fair on Friday, January 2024.
The science fair is hosted at the gymnasium of Hazeldale Elementary
School at 20080 SW Farmington Road in Beaverton Oregon.
"""

In [10]:
def generate(
    response_model, 
    user_prompt, 
    system_prompt="You are a helpful assistant.",
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    # model="gpt-4o-mini",
    # Can use 70b for a higher-quality model
    # model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    max_retries=3,
):
    event = instructor_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt
            },
        ],
        response_model=response_model,
        max_retries=max_retries
    )

    return event

In [11]:
system_prompt = """
Make a calendar event. Respond in JSON with
the event name, date, list of participants,
and the address.
"""

user_prompt = 'Event description: ' + event_description

event = generate(
    CalendarEvent, 
    user_prompt, 
    system_prompt=system_prompt
)

In [12]:
event

CalendarEvent(name='Science Fair', date=datetime.date(2024, 1, 1), participants=[Person(name='Alice'), Person(name='Bob')], address_number='20080', street_name='SW Farmington Road', city_name='Beaverton', state_code='OR', zip_code='97007')

# Retries

In [13]:
from utils import UsageTracker

# Clear any completion response hooks -- prevents 
# duplicate writes to the usage tracker.
instructor_client.clear("completion:response")

# Create a new tracker
tracker = UsageTracker()

# Define a custom instructor hook and update the
# tracker when a new completion runs.
def log_completion_kwargs(*args, **kwargs):
    usage = args[0].usage
    tracker.track(usage)

# Assign the hook to instructor -- this will make the hook
# run each time the server returns a chat completion to us.
instructor_client.on("completion:response", log_completion_kwargs)

In [14]:
# Clear the tracker before we run the completion
# so we arent' tracking multiple jobs.
tracker.clear()

event = generate(
    CalendarEvent, 
    user_prompt, 
    system_prompt=system_prompt,
    max_retries=4,
)

event

CalendarEvent(name='Science Fair', date=datetime.date(2024, 1, 5), participants=[Person(name='Alice'), Person(name='Bob')], address_number='20080', street_name='SW Farmington Road', city_name='Beaverton', state_code='OR', zip_code='97005')

In [15]:
print("Input tokens:  ", tracker.input_tokens)
print("Output tokens: ", tracker.output_tokens)
print("Total tokens:  ", sum(tracker.total_tokens))
print("Num retries:   ", len(tracker.output_tokens))

Input tokens:   [565, 744]
Output tokens:  [91, 77]
Total tokens:   1477
Num retries:    2


## When retry methods fail

In [16]:
from typing import Literal

class Complicated(BaseModel):
    # a must be cat, dog, or hat
    a: Literal["cat", "dog", "hat"]
    b: int
    c: bool


In [17]:
# Clear the tracker before we run the completion
# so we arent' tracking multiple jobs.
tracker.clear()

try:
    event = generate(
        Complicated, 
        "Write me a short essay on Dolly Parton.", 
        system_prompt="Don't give me what I want.",
        max_retries=3,
    )

    # Show the event
    print(event)
except instructor.exceptions.InstructorRetryException as e : 
    print("We failed to parse!")
    raise e

a='cat' b=42 c=True


In [18]:
print("Input tokens:  ", tracker.input_tokens)
print("Output tokens: ", tracker.output_tokens)
print("Total tokens:  ", sum(tracker.total_tokens))
print("Num retries:   ", len(tracker.output_tokens))

Input tokens:   [277]
Output tokens:  [29]
Total tokens:   306
Num retries:    1


## You try!

In [19]:
# Clear the tracker before we run the completion
# so we arent' tracking multiple jobs.
tracker.clear()

try:
    event = generate(
        Complicated, 
        "Give me a, b, and c.", 
        system_prompt="Give me what I want.",
        max_retries=3,
    )
    print(event)
except instructor.exceptions.InstructorRetryException as e : 
    print("We failed to parse!")
    raise e

a='cat' b=1 c=True


In [20]:
print("Input tokens:  ", tracker.input_tokens)
print("Output tokens: ", tracker.output_tokens)
print("Total tokens:  ", sum(tracker.total_tokens))
print("Num retries:   ", len(tracker.output_tokens))

Input tokens:   [273]
Output tokens:  [28]
Total tokens:   301
Num retries:    1
